In [17]:
"""
Generate the mock dwarf galaxy kinematic data exactly as in Nguyen et al. (2022).

Requirements:
- Place star_sampler.py and template.py in your working directory.
- Ensure Python’s import path includes the directory (e.g. export PYTHONPATH=.).
- Install numpy and scipy.

The script below:
1. Samples DM + stellar parameters from the priors (App. A.4).
2. Uses StarSampler’s importance‐sampling to draw (r, vr, vt).
3. Converts to (x,y,z,vx,vy,vz) and projects onto a random line of sight.
4. Adds Gaussian noise (σv=0.1 km/s).
5. Repeats for 80 k training, 10 k validation, 10 k test samples.
6. Saves each split to .npy files.
"""

import numpy as np
import star_sampler as ss   # star_sampler.py
from template import Model  # template.py

def generate_galaxy_sample(params, mu_stars=100, vel_error=0.1, 
                           steps=50, resample_factor=10):
    # 1) draw number of tracer stars
    n_stars = np.random.poisson(mu_stars)
    # 2) initialize the DF model
    model = Model(**params)
    # 3) importance-sample (r, vr, vt) directly
    r_vr_vt_array = ss.impt_sample(
        model_class=model,
        steps=steps,
        resample_factor=resample_factor,
        samplesize=n_stars,
        replace=True,
        r_vr_vt=False  # return original (r, vr, vt) array
    )
    # unpack
    r = r_vr_vt_array[0]
    vr = r_vr_vt_array[1]
    vt = r_vr_vt_array[2]
    # 4) convert to Cartesian (x, y, z, vx, vy, vz)
    x, y, z, vx, vy, vz = ss.r_vr_vt_complete(np.vstack((r, vr, vt)).T)
    # 5) projection to (R, vlos)
    axis = np.random.choice(['x','y','z'])
    if axis == 'x':
        R = np.sqrt(y**2 + z**2); vlos = vx
    elif axis == 'y':
        R = np.sqrt(x**2 + z**2); vlos = vy
    else:  # 'z'
        R = np.sqrt(x**2 + y**2); vlos = vz
    # 6) add measurement noise
    vlos_noisy = vlos + np.random.normal(0, vel_error, size=vlos.shape)
    return R, vlos_noisy, params

def generate_dataset(n_samples, mu_stars=100, vel_error=0.1):
    data, labels = [], []
    for _ in range(n_samples):
        # Priors (App. A.4)
        rho0   = 10**np.random.uniform(5,   8)
        rs     = 10**np.random.uniform(-1,  np.log10(5))
        gamma  = np.random.uniform(-1,     2)
        r_star = np.random.uniform(0.2*rs, 1*rs)
        r_a    = np.random.uniform(0.5*r_star, 2*r_star)
        params = dict(rho0=rho0, rs=rs, gamma=gamma,
                      r_star=r_star, r_a=r_a)
        R, vlos, _ = generate_galaxy_sample(params, mu_stars, vel_error)
        data.append((R, vlos))
        labels.append(params)
    return data, labels

# Generate and save splits
train_data, train_labels = generate_dataset(80000)
val_data,   val_labels   = generate_dataset(10000)
test_data,  test_labels  = generate_dataset(10000) 

np.save('train_data.npy', train_data)
np.save('train_labels.npy', train_labels)
np.save('val_data.npy',   val_data)
np.save('val_labels.npy',   val_labels)
np.save('test_data.npy',  test_data)
np.save('test_labels.npy',  test_labels)

print("Mock datasets generated and saved!")


 
Using importance sampling... 
  building proposal step function... 
  complete proposal function, time used:  27.437026023864746 sec
  start drawing samples...
Weight Variance:  0.13564402288538893 0.6457059869337294
  sampling completed, time used:  0.27136898040771484 sec
sample time:  27.70841908454895 sec
------------------------------------
 
Using importance sampling... 
  building proposal step function... 
  complete proposal function, time used:  26.210996866226196 sec
  start drawing samples...
Weight Variance:  0.14123349883070485 0.4352506260499648
  sampling completed, time used:  0.23663616180419922 sec
sample time:  26.447649002075195 sec
------------------------------------
 
Using importance sampling... 
  building proposal step function... 
  complete proposal function, time used:  21.476460933685303 sec
  start drawing samples...
Weight Variance:  0.1395607255341863 0.3755519089607159
  sampling completed, time used:  0.1662430763244629 sec
sample time:  21.6427350

KeyboardInterrupt: 